IMPORTS ET CHARGEMENTS DES DONNÉES

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

import plotly.express as px

In [2]:
# Chargement des données RFM préalablement calculées et sauvegardées
rfm = pd.read_csv("../data/processed/rfm_segmentation.csv")  # Lecture du fichier CSV nettoyé
# Affichage des 5 premières lignes pour vérification rapide de la structure
rfm.head()

,Customer_ID,Recency,Frequency,Monetary,Cluster
0,12346,165,11,372.86,1
1,12347,3,2,1323.32,0
2,12348,74,1,222.16,0
3,12349,43,3,2671.14,0
4,12351,11,1,300.93,0


DÉFINITION DES FEATURES ET DE LA CIBLE

In [3]:
 # Préparation des variables pour la modélisation prédictive
X = rfm[['Recency', 'Frequency']]   # Variables explicatives (features) :
# - Recency : jours depuis dernier achat
# - Frequency : nombre de commandes
# Objectif : prédire la valeur monétaire comportementale
y = rfm['Monetary']                 # Variable cible (target) :
# Montant total dépensé par le client à prédire

SÉPARATION DES TRAIN/TEST

In [4]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
X,                          # Features : Recency et Frequency
y,                          # Target : Monetary
test_size=0.2,              # 20% des données réservées pour le test, 80% pour l'entraînement
random_state=42             # Reproductibilité : même division à chaque exécution
)

MODÈLE RANDOM FOREST 

In [5]:
# Initialisation et entraînement du modèle de forêt aléatoire
model = RandomForestRegressor(
n_estimators=200,           # Nombre d'arbres dans la forêt (plus = plus précis, plus lent)
random_state=42             # Reproductibilité des résultats
)
# Entraînement du modèle sur les données d'apprentissage
model.fit(X_train, y_train)     # Apprentissage des patterns entre R/F et la valeur monétaire


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

ÉVALUATION DU MODÈLE

In [6]:
# Prédiction et évaluation des performances du modèle
y_pred = model.predict(X_test)          # Prédictions sur l'ensemble de test (données jamais vues)
# Calcul des métriques d'évaluation
mae = mean_absolute_error(y_test, y_pred)   # Erreur absolue moyenne : écart moyen entre prédiction et réalité (en £)
r2 = r2_score(y_test, y_pred)               # Coefficient de détermination : qualité de l'ajustement (0 à 1, 1 = parfait)
# Affichage des résultats
mae, r2

(1266.1507790272994, 0.11160485523358255)

VISUALISATION PRÉDICTION VS RÉALITÉ

In [7]:
# Création d'un DataFrame comparatif entre valeurs réelles et prédites
results = pd.DataFrame({
'Actual': y_test,       # Valeurs réelles du chiffre d'affaires des clients (test)
'Predicted': y_pred     # Valeurs prédites par le modèle Random Forest
})
# Visualisation de la corrélation entre réel et prédit
fig = px.scatter(
results,                        # DataFrame contenant les paires réel/prédit
x='Actual',                     # Axe X : valeurs réelles observées
y='Predicted',                  # Axe Y : valeurs prédites par le modèle
title='Actual vs Predicted Customer Revenue'  # Titre du graphique
)
# Ajout de la ligne de référence parfaite (y = x)
fig.add_shape(
type="line",                    # Type de forme : ligne droite
x0=results['Actual'].min(),     # Point de départ X : minimum des valeurs réelles
y0=results['Actual'].min(),     # Point de départ Y : identique (ligne à 45°)
x1=results['Actual'].max(),     # Point d'arrivée X : maximum des valeurs réelles
y1=results['Actual'].max(),     # Point d'arrivée Y : identique
line=dict(dash="dash")          # Style de ligne : pointillés pour différencier des données
)
# Application du thème visuel épuré
fig.update_layout(template="plotly_white")

fig.write_image("../reports/figures/Actual_VS_Predicted.png")
# Affichage interactif du graphique
fig.show()

IMPORTANCES DES FEATURES

In [8]:
# Analyse de l'importance des variables dans le modèle Random Forest
feature_importance = pd.DataFrame({
'Feature': X.columns,                   # Noms des variables explicatives (Recency, Frequency)
'Importance': model.feature_importances_  # Score d'importance calculé par le modèle (somme = 1)
}).sort_values(
by='Importance',                        # Tri par score d'importance
ascending=False                         # Ordre décroissant : variable la plus importante en premier
)
# Affichage du tableau d'importance des features
feature_importance

,Feature,Importance
1,Frequency,0.856596
0,Recency,0.143404


EXPORT DU MODÉLE (POUR L'APPLICATION)

In [9]:
import joblib

joblib.dump(model, "../models/customer_revenue_model.pkl")

['../models/customer_revenue_model.pkl']

RÉSUMÉ BUSINESS

## Impact commercial

- Le modèle prédit la valeur future des clients en fonction de leur comportement d'achat.

- Il permet de prioriser les clients à forte valeur ajoutée.

- Il facilite l'allocation du budget marketing basée sur les données.